In [ ]:
import json
import os
import re
import logging
from pathlib import Path
from openai import AzureOpenAI
from datetime import datetime

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

# Azure OpenAI configuration
ENDPOINT = "https://dhp-search-east-npe-0.openai.azure.com/"
MODEL = "gpt-4.1"
API_VERSION = "2024-02-01"
AZURE_OPEN_AI_API_KEY = ""

# Initialize the OpenAI client
openai_client = AzureOpenAI(
    api_version=API_VERSION,
    azure_endpoint=ENDPOINT,
    api_key=AZURE_OPEN_AI_API_KEY
)

def write_file(m_code, path):
    """
    Extract code from markdown code block and write to file.
    Enhanced to handle multiple code block formats and provide better error handling.
    
    Args:
        m_code (str): Markdown text containing code blocks
        path (str): Target file path
    
    Returns:
        bool: True if successful, False otherwise
    """
    try:
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(path), exist_ok=True)
        
        # Try multiple patterns for code extraction in order of preference
        
        # Pattern 1: Standard markdown code blocks with language identifier (```jsx, ```python, etc.)
        pattern1 = r"```(?:\w+)?\n(.*?)```"
        match1 = re.search(pattern1, m_code, re.DOTALL)
        
        # Pattern 2: Code blocks with no language identifier (just ```)
        pattern2 = r"```\n(.*?)```"
        match2 = re.search(pattern2, m_code, re.DOTALL)
        
        # Pattern 3: Any code block with or without language identifier
        pattern3 = r"```.*?\n(.*?)```"
        match3 = re.search(pattern3, m_code, re.DOTALL)
        
        # Pattern 4: Import statements as fallback (for identifying code without proper formatting)
        pattern4 = r"(?:^|\n)(import [^\n]+.*?(?:function|class|const|let|var).*?(?:export default|\}))"
        match4 = re.search(pattern4, m_code, re.DOTALL)
        
        # Try each pattern in sequence
        if match1:
            code = match1.group(1).strip()
            logger.info(f"Code extracted using pattern 1 (standard markdown code block)")
        elif match2:
            code = match2.group(1).strip()
            logger.info(f"Code extracted using pattern 2 (simple code block)")
        elif match3:
            code = match3.group(1).strip()
            logger.info(f"Code extracted using pattern 3 (any code block)")
        elif match4:
            code = match4.group(1).strip()
            logger.info(f"Code extracted using pattern 4 (import statements fallback)")
        else:
            # For debugging: Save the raw response to help diagnose the issue
            debug_path = f"{os.path.dirname(path)}/debug_raw_response_{os.path.basename(path).replace('.', '_')}_{datetime.now().strftime('%H%M%S')}.txt"
            with open(debug_path, "w", encoding="utf-8") as f:
                f.write(m_code)
            logger.error(f"No code block found. Raw response saved to {debug_path}")
            
            # Last resort - try to extract any text that looks like code
            # Look for imports or typical code patterns
            potential_code_lines = []
            lines = m_code.split('\n')
            in_code_section = False
            
            for line in lines:
                # Detect start of what might be code
                if ('import ' in line or 'function ' in line or 'class ' in line or 
                    'const ' in line or 'let ' in line or 'var ' in line or 
                    line.strip().startswith('//') or line.strip().startswith('/*')):
                    in_code_section = True
                
                if in_code_section:
                    potential_code_lines.append(line)
            
            if potential_code_lines:
                code = '\n'.join(potential_code_lines)
                logger.warning(f"Used last resort code extraction - extracted {len(potential_code_lines)} lines that look like code")
            else:
                raise ValueError("No code block found in the LLM response")
        
        # Perform framework-specific checks and fixes
        
        # For React files, ensure React is imported if JSX is used
        if path.endswith(('.jsx', '.tsx')) and 'import React' not in code and ('<' in code and '/>' in code):
            code = 'import React from "react";\n\n' + code
            logger.info(f"Added React import to {path} as JSX was detected")
        
        # For React router files, remove BrowserRouter if present
        if 'App.jsx' in path or 'App.tsx' in path:
            if '<BrowserRouter>' in code or '<Router>' in code:
                code = code.replace('<BrowserRouter>', '').replace('</BrowserRouter>', '')
                code = code.replace('<Router>', '').replace('</Router>', '')
                logger.info(f"Removed BrowserRouter/Router from {path} to prevent nesting issues")
        
        # Write code to file
        with open(path, "w", encoding="utf-8") as f:
            f.write(code)
            
        logger.info(f"Successfully wrote {len(code)} characters to {path}")
        return True
    
    except Exception as e:
        logger.error(f"Error writing file {path}: {str(e)}")
        return False

def read_json(path):
    """
    Read and parse JSON file
    
    Args:
        path (str): Path to JSON file
    
    Returns:
        dict: Parsed JSON data
    """
    try:
        with open(path, "r", encoding="utf-8") as f:
            return json.load(f)
    except Exception as e:
        logger.error(f"Error reading JSON file {path}: {str(e)}")
        raise

def write_json_str(json_string, path):
    """
    Write JSON string to file
    
    Args:
        json_string (str): JSON content as string
        path (str): Target file path
    """
    try:
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(path), exist_ok=True)
        
        # Write JSON to file
        with open(path, "w", encoding="utf-8") as f:
            f.write(json_string)
            
        logger.info(f"Successfully wrote JSON to {path}")
    except Exception as e:
        logger.error(f"Error writing JSON file {path}: {str(e)}")
        raise

def load_prompt_template(path: str) -> str:
    """
    Load prompt template from file
    
    Args:
        path (str): Path to prompt template file
    
    Returns:
        str: Content of the prompt template
    """
    try:
        with open(path, "r", encoding="utf-8") as file:
            return file.read()
    except Exception as e:
        logger.error(f"Error loading prompt template {path}: {str(e)}")
        raise

def call_llm(user_instruction: str, system_instruction: str, temperature: float = 0.0) -> str:
    """
    Call LLM with system and user instructions
    
    Args:
        user_instruction (str): User prompt
        system_instruction (str): System prompt
        temperature (float, optional): Temperature for generation. Defaults to 0.0.
    
    Returns:
        str: LLM response
    """
    try:
        start_time = datetime.now()
        logger.info(f"Calling LLM with {len(system_instruction)} chars system prompt and {len(user_instruction)} chars user prompt")
        
        conversation = [
            {"role": "system", "content": system_instruction},
            {"role": "user", "content": user_instruction}
        ]

        response = openai_client.chat.completions.create(
            model=MODEL,
            messages=conversation,
            timeout=600,
            temperature=temperature
        )
        
        elapsed_time = (datetime.now() - start_time).total_seconds()
        logger.info(f"LLM call completed in {elapsed_time:.2f} seconds")
        
        return response.choices[0].message.content
        
    except Exception as e:
        logger.error(f"Error calling LLM: {str(e)}")
        raise

# Web Application Generation Pipeline

This notebook implements an end-to-end pipeline for generating production-ready web applications using AI. The process uses enhanced prompts to create high-quality, structured code for React or Vue applications.

## Enhanced Workflow

### Step 1: Project Planning and Content Generation
- **Project Plan Generation**: Create a comprehensive project plan based on user requirements
- **Page Generation**: Generate each page with enhanced prompts focused on UI/UX, accessibility, and performance
- **Router Generation**: Create routing configuration appropriate for the chosen framework

### Step 2: Project Assembly
- Copy the appropriate base project template (React/Vue with JS/TS)
- Integrate generated pages and router into the project structure
- Ensure proper folder structure and file organization

### Step 3: Project Packaging
- Create a complete, production-ready project directory
- Zip the project for easy distribution
- Add timestamp and descriptive naming for version control

### Step 4: Deployment (Future Enhancement)
- Upload to Azure File Share or other cloud storage
- Generate shareable download links
- Implement deployment automation

## Enhancements Made
- Added robust error handling and logging
- Improved prompt engineering for higher quality outputs
- Created comprehensive documentation
- Implemented proper project packaging and assembly
- Added support for both React and Vue frameworks
- Enhanced code structure and maintainability

---


In [28]:
# Load prompts

router_plan_sys = load_prompt_template("../prompts/router/router-plan-sys.md")
router_gen_user = load_prompt_template("../prompts/router/router-gen-user.md")

css_variables = load_prompt_template("../prompts/css/variables_small.md")
css_selectors = load_prompt_template("../prompts/css/selectors.md")

output_path = "../output"

In [29]:
# Project Plan
framework = "React"
language = "JS"
project_plan_sys = load_prompt_template("../prompts/project/project-plan-sys.md")
project_plan_user = load_prompt_template("../prompts/project/project-plan-user.md").format(FRAMEWORK=framework, LANGUAGE=language, USER_REQUIREMENT="Create a modern online bookstore that allows users to browse, search books.")
project_plan_user

'Create a detailed project plan for a website based on the specified requirements:\n\nFramework: React\nLanguage: JS\nProject Requirements: Create a modern online bookstore that allows users to browse, search books.'

In [30]:
project_plan_json = call_llm(user_instruction=project_plan_user, system_instruction=project_plan_sys)
write_json_str(project_plan_json, f"{output_path}/project/project-plan.json")

In [ ]:
# Page Generation
page_plan = read_json(f"{output_path}/project/project-plan-react-js-multi.json")
page_gen_common_sys = load_prompt_template("../prompts/page/gen/common.md")
page_gen_react_sys = load_prompt_template("../prompts/page/gen/react-only.md")
page_gen_sys_prompt = page_gen_common_sys + page_gen_react_sys
page_gen_user = load_prompt_template("../prompts/page/gen/user.md")

# Add a clear example to the user prompt to guide proper React imports
react_example = """
Here's an example of a properly formatted React component with all necessary imports:

```jsx
import React, { useState, useEffect } from 'react';
import { Link } from 'react-router-dom';
import './ComponentName.css'; // If you have a CSS file

// Import any other components you need
import SomeComponent from '../components/SomeComponent';

function ComponentName() {
  // State and other logic here
  const [data, setData] = useState([]);

  useEffect(() => {
    // Effect logic here
  }, []);

  return (
    <div className="component-name">
      <h1>Component Title</h1>
      {/* JSX content here */}
    </div>
  );
}

export default ComponentName;
```

IMPORTANT: Always include 'import React from "react";' at the top of your component file when using JSX.
"""

print(page_gen_sys_prompt)

logger.info(f"Generating {len(page_plan['pages'])} pages for {page_plan['framework']} {page_plan['language']}")

for page in page_plan["pages"]:
    # Enhance the user prompt with additional examples based on the framework
    enhanced_user_prompt = page_gen_user.format(CSS_VARIABLES=css_variables, CSS_SELECTORS=css_selectors, PROJECT_PLAN=json.dumps(page_plan), PAGE_NAME=page["name"])
    
    # Add React-specific example if using React
    if page_plan["framework"].lower() == "react":
        enhanced_user_prompt += react_example
        
    # Add framework-specific reminder
    if page_plan["framework"].lower() == "react":
        enhanced_user_prompt += "\n\nRemember to include 'import React from \"react\";' at the top of your file!"
        
    # Save the prompt for debugging
    debug_prompt_path = f"{output_path}/debug_page_prompt_{page['name']}.md"
    with open(debug_prompt_path, "w", encoding="utf-8") as f:
        f.write(enhanced_user_prompt)
    logger.info(f"Page generation prompt for {page['name']} saved to {debug_prompt_path}")
    
    # Call LLM with the enhanced prompt
    page_md_code = call_llm(user_instruction=enhanced_user_prompt, system_instruction=page_gen_sys_prompt)
    
    # Save the raw response for debugging
    debug_response_path = f"{output_path}/debug_page_response_{page['name']}.md"
    with open(debug_response_path, "w", encoding="utf-8") as f:
        f.write(page_md_code)
    logger.info(f"Raw page response for {page['name']} saved to {debug_response_path}")
    
    # Write page code to file
    output_page_filepath = f"{output_path}/{page['filepath']}"
    success = write_file(page_md_code, output_page_filepath)
    
    if success:
        logger.info(f"✅ Successfully generated page {page['name']} at {output_page_filepath}")
        
        # For React pages, verify that React is imported if JSX is used
        if page_plan["framework"].lower() == "react":
            with open(output_page_filepath, "r", encoding="utf-8") as f:
                page_content = f.read()
            
            if 'import React' not in page_content and ('<' in page_content and '/>' in page_content):
                logger.warning(f"⚠️ Generated page {page['name']} is missing React import despite using JSX")
                
                # Add React import
                fixed_content = 'import React from "react";\n\n' + page_content
                
                with open(output_page_filepath, "w", encoding="utf-8") as f:
                    f.write(fixed_content)
                
                logger.info(f"✅ Added React import to page {page['name']}")
    else:
        logger.error(f"❌ Failed to generate page {page['name']}")

Your are a senior frontend engineer. Generate a single Page component that is production-ready, minimal and uses the project's custom CSS library (see rules below).
The project was scaffolded with Vite; do not use Node-only globals or APIs.

## Inputs (provided by user)
- `project_plan`: an object that includes the framework, language, brief summary of the site and all the pages.
- `page_name`: the target page to generate. The page's full description must be looked up from `project_plan.pages` by name.
(Use only information found in `project_plan` for this page unlcess otherwise stated.)

## Custom CSS rules
- Prefer the provided custom CSS utilities/classes in the project
- If the page requires styles not covered by the custom CSS:
    - You my define miniimal additional CSS classes yourself, but they must live inside the page file.
    - Do not create or import separate CSS files.
    - Do not use inline `style={...}` / `style="..."` except for truly one-off accessibility fixes.
    

In [ ]:
# Router Generation with Enhanced Prompts
logger.info("Starting router generation process...")

# Extract page routes from the project plan
page_routes = []
for page in page_plan["pages"]:
    page_routes.append({
        "name": page["name"],
        "route": page["route"],
        "filepath": page["filepath"]
    })

# Determine router file path based on framework
if page_plan["framework"].lower() == "react":
    if page_plan["language"].lower() == "ts":
        router_filepath = "src/App.tsx"
    else:
        router_filepath = "src/App.jsx"
else:  # Vue
    if page_plan["language"].lower() == "ts":
        router_filepath = "src/router/index.ts"
    else:
        router_filepath = "src/router/index.js"

# Format the router generation prompt
router_gen_user_formatted = router_gen_user.replace("{{FRAMEWORK}}", page_plan["framework"]) \
                                         .replace("{{LANGUAGE}}", page_plan["language"]) \
                                         .replace("{{PAGE_ROUTES}}", json.dumps(page_routes, indent=2)) \
                                         .replace("{{FILEPATH}}", router_filepath)

# Add a clear example at the end of the prompt to guide the LLM with the correct router structure
if page_plan["framework"].lower() == "react":
    router_gen_user_formatted += """

Please provide the complete code within a standard code block like this:

```jsx
// Example for React App.jsx (IMPORTANT: DO NOT include BrowserRouter!)
import React, { Suspense, lazy } from 'react';
import { Routes, Route, Link, useLocation } from 'react-router-dom';

// Lazy-loaded page components
const HomePage = lazy(() => import('./pages/HomePage'));
const OtherPage = lazy(() => import('./pages/OtherPage'));
// ... other pages

// NotFound component for unmatched routes
const NotFound = () => (
  <div>
    <h1>Page Not Found</h1>
    <p>The page you are looking for does not exist.</p>
    <Link to="/">Return to Home</Link>
  </div>
);

// Navigation component
const Navigation = () => {
  const location = useLocation();
  return (
    <nav>
      <ul>
        <li><Link to="/" className={location.pathname === '/' ? 'active' : ''}>Home</Link></li>
        <li><Link to="/other" className={location.pathname === '/other' ? 'active' : ''}>Other</Link></li>
        {/* Add links for all non-dynamic routes */}
      </ul>
    </nav>
  );
};

function App() {
  return (
    <div className="app">
      <header>
        <Navigation />
      </header>
      <main>
        <Suspense fallback={<div>Loading...</div>}>
          <Routes>
            <Route path="/" element={<HomePage />} />
            <Route path="/other" element={<OtherPage />} />
            {/* Add routes for all pages */}
            <Route path="*" element={<NotFound />} />
          </Routes>
        </Suspense>
      </main>
    </div>
  );
}

export default App;
```

IMPORTANT NOTE: For App.jsx/tsx, DO NOT include a BrowserRouter component! The main.jsx/tsx will already provide that. Using nested Router components will cause this error: "You cannot render a <Router> inside another <Router>".
"""
else:
    # Vue router example
    router_gen_user_formatted += """

Please provide the complete code within a standard code block like this:

```js
// Example for Vue router
import { createRouter, createWebHistory } from 'vue-router';

// Route definitions
const routes = [
  {
    path: '/',
    name: 'Home',
    component: () => import('../pages/HomePage.vue')
  },
  {
    path: '/about',
    name: 'About',
    component: () => import('../pages/AboutPage.vue')
  },
  // Add routes for all pages
  {
    path: '/:pathMatch(.*)*',
    name: 'NotFound',
    component: () => import('../pages/NotFoundPage.vue')
  }
];

const router = createRouter({
  history: createWebHistory(),
  routes
});

export default router;
```

Make sure the code is properly formatted and enclosed in triple backticks.
"""

# Generate router code
logger.info(f"Generating router for {page_plan['framework']} {page_plan['language']} with {len(page_routes)} routes")

# Save the prompt for debugging if needed
debug_prompt_path = f"{output_path}/debug_router_prompt.md"
with open(debug_prompt_path, "w", encoding="utf-8") as f:
    f.write(router_gen_user_formatted)
logger.info(f"Router generation prompt saved to {debug_prompt_path}")

# Call LLM with slightly higher temperature for creativity
router_code = call_llm(
    user_instruction=router_gen_user_formatted,
    system_instruction=router_plan_sys,
    temperature=0.2  # Slightly higher temperature for code generation
)

# Save the raw response for debugging
debug_response_path = f"{output_path}/debug_router_response.md"
with open(debug_response_path, "w", encoding="utf-8") as f:
    f.write(router_code)
logger.info(f"Raw router response saved to {debug_response_path}")

# Write router code to file
output_router_filepath = f"{output_path}/{router_filepath}"
success = write_file(router_code, output_router_filepath)

if success:
    logger.info(f"✅ Successfully generated router at {output_router_filepath}")
    
    # For React apps, verify that BrowserRouter is not included in the App.jsx/tsx
    if page_plan["framework"].lower() == "react":
        with open(output_router_filepath, "r", encoding="utf-8") as f:
            app_content = f.read()
        
        if "BrowserRouter" in app_content:
            logger.warning("⚠️ Generated App component contains BrowserRouter which may cause nested router errors")
            
            # Replace BrowserRouter with a fragment
            fixed_content = app_content.replace("<BrowserRouter>", "").replace("</BrowserRouter>", "")
            
            with open(output_router_filepath, "w", encoding="utf-8") as f:
                f.write(fixed_content)
            
            logger.info("✅ Removed BrowserRouter from App component to prevent nested router errors")
else:
    logger.error(f"❌ Failed to generate router")
    
    # Attempt to extract code using a more flexible approach if normal extraction failed
    try:
        # Try a more flexible regex to find any code-like content
        code_pattern = r"```[^`]*```|`[^`]+`"
        matches = re.findall(code_pattern, router_code, re.DOTALL)
        
        if matches:
            longest_match = max(matches, key=len)
            # Clean up the code by removing backticks
            clean_code = re.sub(r"^```\w*\n|```$", "", longest_match).strip()
            
            # For React apps, ensure BrowserRouter is not included
            if page_plan["framework"].lower() == "react" and "BrowserRouter" in clean_code:
                clean_code = clean_code.replace("<BrowserRouter>", "").replace("</BrowserRouter>", "")
                logger.info("✅ Removed BrowserRouter from extracted code to prevent nested router errors")
            
            # Write the extracted code
            os.makedirs(os.path.dirname(output_router_filepath), exist_ok=True)
            with open(output_router_filepath, "w", encoding="utf-8") as f:
                f.write(clean_code)
            logger.info(f"✅ Recovered and wrote router code using fallback method")
        else:
            logger.error("Could not extract any code from the response even with fallback method")
    except Exception as e:
        logger.error(f"Error in fallback code extraction: {str(e)}")

In [ ]:
# Project Packaging and Finalization
import shutil
import zipfile
from datetime import datetime

def copy_base_project(framework, language, destination):
    """
    Copy the appropriate base project to the destination
    
    Args:
        framework (str): 'react' or 'vue'
        language (str): 'js' or 'ts'
        destination (str): Destination directory
    
    Returns:
        bool: Success status
    """
    framework = framework.lower()
    language = language.lower()
    
    # Determine source directory
    if framework == "react":
        if language == "ts":
            source_dir = "../base_projects/react-ts"
        else:
            source_dir = "../base_projects/react-js"
    elif framework == "vue":
        if language == "ts":
            source_dir = "../base_projects/vue-ts"
        else:
            source_dir = "../base_projects/vue-js"
    else:
        logger.error(f"Unsupported framework: {framework}")
        return False
    
    try:
        # Copy the base project
        if os.path.exists(destination):
            shutil.rmtree(destination)
        
        shutil.copytree(source_dir, destination)
        logger.info(f"Base project copied from {source_dir} to {destination}")
        return True
    except Exception as e:
        logger.error(f"Failed to copy base project: {str(e)}")
        return False

def create_project_zip(project_dir, output_zip):
    """
    Create a zip file of the project
    
    Args:
        project_dir (str): Source project directory
        output_zip (str): Output zip file path
    
    Returns:
        bool: Success status
    """
    try:
        with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for root, _, files in os.walk(project_dir):
                for file in files:
                    file_path = os.path.join(root, file)
                    rel_path = os.path.relpath(file_path, project_dir)
                    zipf.write(file_path, rel_path)
        
        logger.info(f"Project zip created at {output_zip}")
        return True
    except Exception as e:
        logger.error(f"Failed to create project zip: {str(e)}")
        return False

# Create final project directory
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
project_name = f"bookstore_{page_plan['framework'].lower()}_{page_plan['language'].lower()}"
final_project_dir = f"{output_path}/final/{project_name}_{timestamp}"
logger.info(f"Creating final project at {final_project_dir}")

# Copy base project
if copy_base_project(page_plan["framework"], page_plan["language"], final_project_dir):
    logger.info("Base project copied successfully")
    
    # Copy generated pages
    os.makedirs(f"{final_project_dir}/src/pages", exist_ok=True)
    logger.info("Copying generated pages to final project...")
    
    for page in page_plan["pages"]:
        source_file = f"{output_path}/{page['filepath']}"
        dest_file = f"{final_project_dir}/{page['filepath']}"
        
        if os.path.exists(source_file):
            os.makedirs(os.path.dirname(dest_file), exist_ok=True)
            shutil.copy2(source_file, dest_file)
            logger.info(f"Copied {page['name']} to {dest_file}")
    
    # Copy router file
    router_rel_path = router_filepath  # From the router generation cell
    source_router = f"{output_path}/{router_rel_path}"
    dest_router = f"{final_project_dir}/{router_rel_path}"
    
    if os.path.exists(source_router):
        os.makedirs(os.path.dirname(dest_router), exist_ok=True)
        shutil.copy2(source_router, dest_router)
        logger.info(f"Copied router to {dest_router}")
    
    # Create zip file
    zip_file = f"{output_path}/final/{project_name}_{timestamp}.zip"
    if create_project_zip(final_project_dir, zip_file):
        logger.info(f"✅ Project successfully packaged and zipped")
        logger.info(f"Final project: {final_project_dir}")
        logger.info(f"Zip file: {zip_file}")
    else:
        logger.error("Failed to create project zip")
else:
    logger.error("Failed to copy base project")